# WatsonX.ai with Milvus and LangChain

This guide demonstrates how to build an Watsonx.ai LLM-driven question-answering application with Milvus and LangChain

## Set up the environment
Before you use the sample code in this notebook, you must perform the following setup tasks:

Create a Watson [Machine Learning (WML)](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) Service instance (a free plan is offered and information about how to create the instance can be found [here](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics)).

### Install and import dependecies

In [1]:
from IPython.display import clear_output
!pip install pymilvus
!pip install "langchain==0.0.345" 
!pip install wget 
!pip install sentence-transformers 
!pip install chromadb==0.3.22 
!pip install "ibm-watson-machine-learning>=1.0.335" 
!pip install "pydantic>=1.4.0,<2" 
!pip install bs4
!pip install ipywidgets
!pip install towhee towhee.models 
!pip install gradio "pydantic>=1.4.0,<2" datasets
clear_output()


### Setup Remote Server
Here we should define the variable `REMOTE_SERVER` just created [here](https://github.com/ruslanmv/Watsonx-Assistant-with-Milvus-as-Vector-Database/blob/master/README.md)

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
COLLECTION_NAME = 'qa_medical'
DIMENSION = 768
MILVUS_PORT = "19530"
REMOTE_SERVER = os.environ.get("REMOTE_SERVER", "localhost")

### Connect to a Milvus server

In [3]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
conn = connections.connect(host=REMOTE_SERVER, port=19530)

### Create database

In [4]:
from pymilvus import connections, db
db.drop_database("medical")
database = db.create_database("medical")

 List databases

In [5]:
db.list_database()

['default', 'medical']

If the collection already exists, drop it.

In [6]:
from pymilvus import utility
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

List all collections

In [7]:
from pymilvus import utility
utility.list_collections()

[]

### Prepare the Data

In [8]:
from datasets import load_dataset
from IPython.display import clear_output
dataset = load_dataset("ruslanmv/ai-medical-chatbot")
clear_output()

**dataset**: a file containing question and the answer.

Let's take a quick look:

In [9]:
column_names = dataset["train"].column_names
print(column_names)

['Description', 'Patient', 'Doctor']


In [10]:
train_data = dataset["train"]
for i in range(1):
    print(train_data[i])

{'Description': 'Q. What does abutment of the nerve root mean?', 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?', 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}


For this demo let us choose the first 1000 dialogues

In [11]:
import pandas as pd
df = pd.DataFrame(train_data[:1000])

In [12]:
df.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


For the development of the model, let just consider the patient and doctor

In [13]:
#df = df[["Patient", "Doctor"]].rename(columns={"Patient": "question", "Doctor": "answer"})
df = df[["Description", "Doctor"]].rename(columns={"Description": "question", "Doctor": "answer"})

In [14]:
# Add the 'ID' column as the first column
df.insert(0, 'id', df.index)
# Reset the index and drop the previous index column
df = df.reset_index(drop=True)

In [15]:
import re
# Clean the 'question' and 'answer' columns
df['question'] = df['question'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
df['answer'] = df['answer'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
df['question'] = df['question'].str.replace('^Q.', '', regex=True)
# Assuming your DataFrame is named df
max_length = 500  # Due to our enbeeding model does not allow long strings
df['question'] = df['question'].str.slice(0, max_length)

In [16]:
df.head()

,id,question,answer
0,0,What does abutment of the nerve root mean?,Hi. I have gone through your query with dilige...
1,1,What should I do to reduce my weight gained d...,Hi. You have really done well with the hypothy...
2,2,I have started to get lots of acne on my face...,Hi there Acne has multifactorial etiology. Onl...
3,3,Why do I have uncomfortable feeling between t...,Hello. The popping and discomfort what you fel...
4,4,My symptoms after intercourse threatns me eve...,Hello. The HIV test uses a finger prick blood ...


To use the dataset to get answers, let's first define the dictionary:

- `id_answer`: a dictionary of id and corresponding answer

In [17]:
id_answer = df.set_index('id')['answer'].to_dict()


### Create Milvus Collection

Before getting started, please make sure that you have started a [Milvus service](https://milvus.io/docs/install_standalone-docker.md). This notebook uses [milvus 2.2.10](https://milvus.io/docs/v2.2.x/install_standalone-docker.md) and [pymilvus 2.3.6](https://milvus.io/docs/release_notes.md#2360).

In [18]:
!python -m pip install -q pymilvus==2.3.6

Next to define the function `create_milvus_collection` to create collection in Milvus that uses the [L2 distance metric](https://milvus.io/docs/metric.md#Euclidean-distance-L2) and an [IVF_FLAT index](https://milvus.io/docs/index.md#IVF_FLAT).

### Setup Remote Server
Here we should define the variable `REMOTE_SERVER` just created [here](https://github.com/ruslanmv/Watsonx-Assistant-with-Milvus-as-Vector-Database/blob/master/README.md)

In [19]:
from dotenv import load_dotenv
import os
load_dotenv()
host_milvus = os.environ.get("REMOTE_SERVER", '127.0.0.1')
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
connections.connect(host=host_milvus, port='19530')

In [20]:
collection_name=COLLECTION_NAME
def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)

    fields = [
    FieldSchema(name='id', dtype=DataType.INT64, descrition='ids', max_length=500, is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='reverse image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection
collection = create_milvus_collection(collection_name, 768)

### Load question embedding into Milvus

We first generate embedding from question text with [dpr](https://towhee.io/text-embedding/dpr) operator and insert the embedding into Milvus. Towhee provides a [method-chaining style API](https://towhee.readthedocs.io/en/main/index.html) so that users can assemble a data processing pipeline with operators.

In [21]:
from IPython.display import clear_output

In [22]:
%%time
from towhee import pipe, ops
import numpy as np
from towhee.datacollection import DataCollection
from IPython.display import clear_output
max_input_length = 500  # Maximum length allowed by the model
insert_pipe = (
    pipe.input('id', 'question', 'answer')
        .map('question', 'vec', lambda x: x[:max_input_length])  # Truncate the question if longer than 500 tokens
        .map('vec', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map(('id', 'vec'), 'insert_status', ops.ann_insert.milvus_client(host=host_milvus, port='19530', collection_name=collection_name))
        .output()
)
clear_output()  

CPU times: user 859 ms, sys: 906 ms, total: 1.77 s
Wall time: 7.7 s


In [23]:
%%time
# Assuming you have a DataFrame named df
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    question = row['question']
    # Truncate the question string if it exceeds the expected size for the model
    if len(question) > 500:
        row['question'] = question[:500]  # Truncate the question string if it exceeds 500 characters
    insert_pipe(*row)
# Clear the output
clear_output()    

CPU times: user 12.4 s, sys: 656 ms, total: 13 s
Wall time: 2min 48s


In [24]:
from pymilvus import utility
if utility.has_collection(COLLECTION_NAME):
    print("There is  a Collection: " + COLLECTION_NAME )
    
from pymilvus import utility
utility.list_collections()

There is  a Collection: qa_medical


['qa_medical']

## Load a Collection
Milvus allows users to load a collection as multiple replicas to utilize the CPU and memory resources of extra query nodes. This feature boosts the overall QPS and throughput without extra hardware. Before loading a collection, ensure that you have already indexed it.

In [25]:
from pymilvus import Collection, utility
collection = Collection(COLLECTION_NAME)      
collection.load(replica_number=1)
utility.load_state(COLLECTION_NAME)
utility.loading_progress(COLLECTION_NAME)


{'loading_progress': '100%'}

## Prepare search parameters
Prepare the parameters that suit your search scenario. The following example defines that the search will calculate the distance with Euclidean distance, and retrieve vectors from ten closest clusters built by the IVF_FLAT index.

In [26]:
search_params = {
    "metric_type": "L2", 
    "offset": 0, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}

Milvus supports setting consistency level specifically for a search. The example in this topic sets the consistency level as Strong. You can also set the consistency level as Bounded, Session or Eventually. 

In [27]:
from towhee import pipe, ops
import numpy as np
def convert_text_to_embedding(input_text):
    # Define the pipeline for text embedding, ensuring float32 output
    embedding_pipe = (
        pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
        .map('vec', 'normalized_vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('normalized_vec', 'float_vec', lambda x: np.array(x, dtype=np.float32))  # Explicitly convert to float32
        .output('float_vec')
    )
    # Get the embedding for the input text
    embedding_queue = embedding_pipe(input_text)
    embedding = embedding_queue.get()
    return embedding[0]  # Return the first (and only) embedding

def search_milvus_collection(collection, query_embedding, top_k=10):
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    results = collection.search(data=[query_embedding], anns_field="embedding", param=search_params, limit=top_k)
    return results
# Example usage
input_text = "I have started to get lots of acne on my face, particularly on my forehead. Please help me"
query_embedding = convert_text_to_embedding(input_text)

# Search the collection
search_results = search_milvus_collection(collection, query_embedding)


In [28]:
# get the IDs of all returned hits
search_results[0].ids

[2, 835, 156, 180, 381, 333, 298, 448, 663, 834]

In [29]:
# get the distances to the query vector from all returned hits
search_results[0].distances

[0.02460573986172676,
 0.31883323192596436,
 0.35052570700645447,
 0.383177787065506,
 0.3857323229312897,
 0.38954097032546997,
 0.3918834328651428,
 0.39505255222320557,
 0.4189419746398926,
 0.4191533029079437]

In [30]:
id_answer = df.set_index('id')['answer'].to_dict()

In [31]:
def print_search_results_with_answers(results, id_answer, top_n=5):
    # Sort the results by distance (ascending order)
    sorted_results = sorted(results[0], key=lambda x: x.distance)
    
    # Limit the number of displayed results
    displayed_results = sorted_results[:top_n]
    
    for idx, result in enumerate(displayed_results, start=1):
        print(f"Result {idx}:")
        print(f"  ID: {result.id}")
        print(f"  Distance: {result.distance:.4f}")
        answer_text = id_answer.get(result.id, 'Answer not found')
        print(f"  Answer: {answer_text}")
        print()

# Print the top 5 search results sorted by distance
print_search_results_with_answers(search_results, id_answer, top_n=1)

Result 1:
  ID: 2
  Distance: 0.0246
  Answer: Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash



Now that embedding for question dataset have been inserted into Milvus, we can ask question with Milvus and Towhee. Again, we use Towhee to load the input question, compute a embedding, and use it as a query in Milvus. Because Milvus only outputs IDs and distance values, we provide the `id_answers` dictionary to get the answers based on IDs and display.

In [32]:
from towhee import pipe, ops
import numpy as np
from towhee.datacollection import DataCollection
from IPython.display import clear_output
# Define the maximum input length for the question
max_input_length = 512

In [33]:
%%time
# Load the collection
collection.load()
# Create the combined pipe for question encoding and answer retrieval
combined_pipe = (
    pipe.input('question')
        .map('question', 'vec', lambda x: x[:max_input_length])  # Truncate the question if longer than 512 tokens
        .map('vec', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('vec', 'res', ops.ann_search.milvus_client(host=host_milvus, port='19530', collection_name=collection_name, limit=1))
        .map('res', 'answer', lambda x: [id_answer[int(i[0])] for i in x])
        .output('question', 'answer')
)
 

CPU times: user 62.5 ms, sys: 141 ms, total: 203 ms
Wall time: 1.67 s


In [34]:
%%time
# Perform the encoding and retrieval for a specific question
ans = combined_pipe('I have started to get lots of acne on my face, particularly on my forehead. Please help me')
ans = DataCollection(ans)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 416 ms


In [35]:
ans.show()

question,answer
"I have started to get lots of acne on my face, particularly on my forehead. Please help me",Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to h...


In [36]:
ans[0]['answer']

['Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash']

## LangChain with Milvus Collection

In [37]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.vectorstores.milvus import Milvus
from langchain.embeddings import HuggingFaceEmbeddings  # Not used in this example
# (Assuming you have the required SentenceTransformer model loaded)
embeddings = SentenceTransformerEmbeddings()  # Create SentenceTransformer embeddings instance


In [38]:
COLLECTION_NAME='qa_medical'
from dotenv import load_dotenv
import os
load_dotenv()
host_milvus = os.environ.get("REMOTE_SERVER", '127.0.0.1')
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
connections.connect(host=host_milvus, port='19530')

In [39]:
from pymilvus import Collection, utility
collection = Collection(COLLECTION_NAME)      
collection.load(replica_number=1)
utility.load_state(COLLECTION_NAME)
utility.loading_progress(COLLECTION_NAME)


{'loading_progress': '100%'}

In [40]:
from towhee import pipe, ops
import numpy as np
def convert_text_to_embedding(input_text):
    # Define the pipeline for text embedding, ensuring float32 output
    embedding_pipe = (
        pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
        .map('vec', 'normalized_vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('normalized_vec', 'float_vec', lambda x: np.array(x, dtype=np.float32))  # Explicitly convert to float32
        .output('float_vec')
    )
    # Get the embedding for the input text
    embedding_queue = embedding_pipe(input_text)
    embedding = embedding_queue.get()
    return embedding[0]  # Return the first (and only) embedding

In [41]:
query = "I have started to get lots of acne on my face, particularly on my forehead. Please help me"
query_embedding = convert_text_to_embedding(query)  # Get embedding for the query sentence

In [42]:
def search_milvus_collection(collection, query_embedding, top_k=10):
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    results = collection.search(data=[query_embedding], anns_field="embedding", param=search_params, limit=top_k)
    return results

In [43]:
# Search the collection
search_results = search_milvus_collection(collection, query_embedding)

In [44]:

docs = [result.ids[0] for result in search_results]  # Get IDs of matching documents (assuming single field)


In [45]:
def print_search_results_with_answers(results, id_answer, top_n=5):
    # Sort the results by distance (ascending order)
    sorted_results = sorted(results[0], key=lambda x: x.distance)
    # Limit the number of displayed results
    displayed_results = sorted_results[:top_n]
    for idx, result in enumerate(displayed_results, start=1):
        print(f"Result {idx}:")
        print(f"  ID: {result.id}")
        print(f"  Distance: {result.distance:.4f}")
        answer_text = id_answer.get(result.id, 'Answer not found')
        print(f"  Answer: {answer_text}")
# Print the top 5 search results sorted by distance
print_search_results_with_answers(search_results, id_answer, top_n=1)

Result 1:
  ID: 2
  Distance: 0.0246
  Answer: Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash


In [46]:
def collect_retrieved_docs(search_results, num_docs=10, sort_key="distance", ascending=True):
  """
  Collects and sorts retrieved documents from the search results.

  Args:
    search_results: The output of a Milvus search result, typically a list of objects.
    num_docs: The maximum number of documents to collect (default: 10).
    sort_key: The key to sort documents by (default: "distance"). Can be "id", "distance", or custom field names.
    ascending: Whether to sort in ascending order (default: True).

  Returns:
    A list of retrieved documents (either as objects or their IDs).
  """
  # Extract document IDs and distance values
  retrieved_data = [(result.id, result.distance) for result in search_results[0]]

  # Sort by specified key and order
  sorted_data = sorted(retrieved_data, key=lambda x: x[1] if sort_key == "distance" else x[0], reverse=not ascending)

  # Select and return the desired number of documents/IDs
  if num_docs > 0:
    return [item[0] for item in sorted_data[:num_docs]]
  else:
    return [item for item in sorted_data]


In [47]:
docs=collect_retrieved_docs(search_results, num_docs=2)

In [48]:
docs

[2, 835]

In [49]:
def collect_retrieved_docs(search_results, num_docs=10, sort_key="distance", ascending=True, id_answer=None):
  """
  Collects, sorts, and extracts text for retrieved documents.

  Args:
    search_results: The output of a Milvus search result.
    num_docs: The maximum number of documents to collect (default: 10).
    sort_key: The key to sort documents by (default: "distance").
    ascending: Whether to sort in ascending order (default: True).
    id_answer: A dictionary mapping document IDs to their text (optional).

  Returns:
    A list of dictionaries, each containing "id" and "text" keys for the retrieved documents.
  """
  retrieved_data = [(result.id, result.distance) for result in search_results[0]]
  sorted_data = sorted(retrieved_data, key=lambda x: x[1] if sort_key == "distance" else x[0], reverse=not ascending)

  retrieved_docs = []
  for item in sorted_data[:num_docs]:
    doc_id = item[0]
    doc_text = id_answer.get(doc_id, 'Answer not found')  # Retrieve text from the dictionary
    retrieved_docs.append({"id": doc_id, "text": doc_text})

  return retrieved_docs

In [50]:
docs = collect_retrieved_docs(search_results, num_docs=2, id_answer=id_answer)

In [51]:
docs[0]['text']

'Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash'

In [52]:
for doc in docs:
  print(f"Document ID: {doc['id']}")
  print(f"Document Text: {doc['text']}")

Document ID: 2
Document Text: Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash
Document ID: 835
Document Text: Hi. Well, ideally if you have bilateral inguinal hernia, or inguinaĺ hernia on one side and hydrocele on the other, the surgery should be done for one first before the other, not simultaneously so that when you are fully recovered of one, then the other can be planned and done. Pain can be due to entrapment of a nerve like ileoinguinal or ileohypogastric nerves which run with the spermatic cords or it could be also present if there is improper repair with subsequent break

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation Model inferencing.
Action: Provide the IBM Cloud user API key. For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

Hint: You can find the project_id as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be Projects / <project name> /. Click on the <project name> link. Then get the project_id from Project's Manage tab (Project -> Manage -> General -> Details).


In [53]:
from dotenv import load_dotenv
import os
load_dotenv()
try:
    API_KEY = os.environ.get("API_KEY")
    project_id =os.environ.get("PROJECT_ID")
except KeyError:
    API_KEY: input("Please enter your WML api key (hit enter): ")
    project_id  = input("Please  project_id (hit enter): ")

In [54]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY  
}

## Foundation Models on watsonx
### Defining model
You need to specify model_id that will be used for inferencing:



In [55]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
model_id = ModelTypes.GRANITE_13B_CHAT_V2

## Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [56]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

LangChain CustomLLM wrapper for watsonx model
Initialize the WatsonxLLM class from Langchain with defined parameters and ibm/granite-13b-chat-v2

In [57]:
from langchain.llms import WatsonxLLM
watsonx_granite = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

Generate a retrieval-augmented response to a question
Build the RetrievalQA (question answering chain) to automate the RAG task.


In [58]:
def collect_retrieved(search_results, num_docs=10, sort_key="distance", ascending=True, id_answer=None):
  """
  Collects, sorts, and extracts text for retrieved documents.

  Args:
    search_results: The output of a Milvus search result.
    num_docs: The maximum number of documents to collect (default: 10).
    sort_key: The key to sort documents by (default: "distance").
    ascending: Whether to sort in ascending order (default: True).
    id_answer: A dictionary mapping document IDs to their text (optional).

  Returns:
    A list of dictionaries, each containing "id" and "text" keys for the retrieved documents.
  """
  retrieved_data = [(result.id, result.distance) for result in search_results[0]]
  sorted_data = sorted(retrieved_data, key=lambda x: x[1] if sort_key == "distance" else x[0], reverse=not ascending)

  retrieved_docs = []
  for item in sorted_data[:num_docs]:
    doc_id = item[0]
    doc_text = id_answer.get(doc_id, 'Answer not found')  # Retrieve text from the dictionary
    retrieved_docs.append({"id": doc_id, "text": doc_text})

  return retrieved_docs

import langchain.chains as lc
# Assuming you have initialized watsonx_granite as the LLM
query = "I have started to get lots of acne on my face, particularly on my forehead what can I do"
# Collect top 2 documents sorted by descending relevance
docs = collect_retrieved(search_results, num_docs=2, id_answer=id_answer)

In [86]:
docs

[{'id': 2,
  'text': 'Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash'},
 {'id': 835,
  'text': 'Hi. Well, ideally if you have bilateral inguinal hernia, or inguinaĺ hernia on one side and hydrocele on the other, the surgery should be done for one first before the other, not simultaneously so that when you are fully recovered of one, then the other can be planned and done. Pain can be due to entrapment of a nerve like ileoinguinal or ileohypogastric nerves which run with the spermatic cords or it could be also present if there is improper repair with subsequent breakdown of the r

'Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash'

In [79]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List
# Load the collection
collection.load()
# Create the combined pipe for question encoding and answer retrieval
combined_pipe = (
    pipe.input('question')
        .map('question', 'vec', lambda x: x[:max_input_length])  # Truncate the question if longer than 512 tokens
        .map('vec', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('vec', 'res', ops.ann_search.milvus_client(host=host_milvus, port='19530', collection_name=collection_name, limit=1))
        .map('res', 'answer', lambda x: [id_answer[int(i[0])] for i in x])
        .output('question', 'answer')
)
class CustomRetriever(BaseRetriever): 
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        # Perform the encoding and retrieval for a specific question
        ans = combined_pipe(query)
        ans = DataCollection(ans)
        answer=ans[0]['answer']
        answer_string = ' '.join(answer)
        return [Document(page_content=answer_string)]
retriever = CustomRetriever(documents=docs)
query = "I have started to get lots of acne on my face, particularly on my forehead what can I do"
relevant_documents = retriever.get_relevant_documents(query)
print(relevant_documents)    

[Document(page_content='Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash')]


In [80]:
# This will only documents related with the query
query = "I have started to get lots of acne on my face, particularly on my forehead what can I do"
docs_search =retriever.get_relevant_documents(query)

In [81]:
docs_search

[Document(page_content='Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash')]

In [83]:
retriever

CustomRetriever()

In [84]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=watsonx_granite,
                                  chain_type="stuff", 
                                  retriever=retriever)


## Ask your question
After preparing the documents, you can set up a chain to include them in a prompt. This will allow LLM to use the docs as a reference when preparing answers.
Get questions from the previously loaded dataset.

In [85]:
query = "I have started to get lots of acne on my face, particularly on my forehead what can I do"
qa.run(query)

"\nI'm not a dermatologist, but I can provide some general advice. For mild acne, a gentle face wash and a topical medication like clindamycin or retenoic acid can be helpful. However, if your acne is more severe (grade 2 or higher), you will likely need oral medication in addition to topical treatments. It's important to consult with a dermatologist to get an accurate diagnosis and treatment plan. They may recommend antibiotics, retinoids, or other medications based on your specific"

In [ ]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
rag_prompt = PromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | watsonx_granite
)

In [ ]:
print(rag_chain.invoke("Explain IVF_FLAT in Milvus."))



 IVF_FLAT is an index type in Milvus that divides vector space into list clusters. At the default list value of 16,384, Milvus compares the distances between the target vector and the centroids of all 16,384 clusters to return probe nearest clusters. Then Milvus compares the distances between the target vector and the vectors in the selected clusters to get the nearest vectors. This method can be time-consuming when dealing with a large number of vectors.


